# Cargar CSV a Base de Datos Spotify

In [72]:
pip install pandas mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [73]:
import pandas as pd
import mysql.connector

# Cargar los archivos
df_artistas_info = pd.read_csv('spotify_base_datos_info_artistas_todos.csv')
df_canciones = pd.read_csv('spotify_base_datos_canciones_todas.csv')

# Eliminar duplicados de artistas para evitar errores de inserción
df_artistas_info = df_artistas_info.drop_duplicates(subset=['artista'])

In [74]:
# Conectar con tu servidor MySQL.

def conectar():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Amsterdam19",
        database="bd_spotify"
    )

conexion = conectar()
cursor = conexion.cursor()

In [75]:
# Extraemos los géneros únicos de la tabla de canciones e insertamos en la tabla generos
generos_unicos = df_canciones['genre'].unique()

for genero in generos_unicos:
    try:
        cursor.execute("INSERT IGNORE INTO generos (nombre_genero) VALUES (%s)", (genero,))
    except Exception as e:
        print(f"Error insertando género {genero}: {e}")

conexion.commit()
print("Géneros insertados.")

Géneros insertados.


In [76]:
#Informacion detallada de los artistas

for _, row in df_artistas_info.iterrows():
    sql = """INSERT INTO artistas (nombre, biografia, oyentes, num_reproducciones, artistas_similares) 
             VALUES (%s, %s, %s, %s, %s)"""
    valores = (row['artista'], row['biografia'], row['Oyentes'], row['N_Reproducciones'], row['artistas_similares'])
    
    try:
        cursor.execute(sql, valores)
    except Exception as e:
        print(f"Error con artista {row['artista']}: {e}")

conexion.commit()
print("Artistas insertados.")

Artistas insertados.


In [77]:
# Cambiamos el atributo INT en la tabla de artistas a BIGINT para que nos salgan las reproducciones de Taylor Swift

# --- REPARACIÓN DE LA TABLA ---
try:
    print("Reparando columna para Taylor Swift...")
    cursor.execute("ALTER TABLE artistas MODIFY COLUMN num_reproducciones BIGINT;")
    conexion.commit()
    print("✅ Capacidad aumentada a BIGINT.")
except Exception as e:
    print(f"Nota: No se pudo modificar (quizás ya era BIGINT): {e}")



Reparando columna para Taylor Swift...
✅ Capacidad aumentada a BIGINT.


In [78]:
# 1. Crear mapas de Nombres -> IDs
cursor.execute("SELECT id_genero, nombre_genero FROM generos")
mapa_generos = {nombre: id_gen for (id_gen, nombre) in cursor.fetchall()}

cursor.execute("SELECT id_artista, nombre FROM artistas")
mapa_artistas = {nombre: id_art for (id_art, nombre) in cursor.fetchall()}

# 2. Insertar canciones mapeando los nombres a sus IDs
for _, row in df_canciones.iterrows():
    # Buscamos los IDs correspondientes
    id_gen = mapa_generos.get(row['genre'])
    id_art = mapa_artistas.get(row['name_artist'])
    
    # Solo insertamos si encontramos al artista (para mantener integridad)
    if id_art and id_gen:
        sql = "INSERT INTO canciones (nombre, anio, id_genero, id_artista) VALUES (%s, %s, %s, %s)"
        cursor.execute(sql, (row['name_track'], row['year'], id_gen, id_art))

conexion.commit()
cursor.close()
print("Proceso finalizado con éxito.")

Proceso finalizado con éxito.


In [79]:
# Artistas:

# --- 1. ¿Qué artista tiene más canciones publicadas? ---

query1 = """
SELECT a.nombre, COUNT(c.id_artista) AS total_canciones
FROM artistas a
JOIN canciones c ON a.id_artista = c.id_artista
GROUP BY a.nombre
ORDER BY total_canciones DESC
LIMIT 1;
"""
print("\n1. Artista con más canciones:")
print(pd.read_sql(query1, conexion))


1. Artista con más canciones:
        nombre  total_canciones
0  Tame Impala              528


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\4270660085.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query1, conexion))


In [80]:
# --- 2. ¿Qué artista tiene más reproducciones totales? ---

query2 = """
SELECT nombre, num_reproducciones 
FROM artistas 
ORDER BY num_reproducciones DESC 
LIMIT 1;
"""
print("\n2. Artista con más reproducciones totales:")
print(pd.read_sql(query2, conexion))


2. Artista con más reproducciones totales:
         nombre  num_reproducciones
0  Taylor Swift          3497593576


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2502882892.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query2, conexion))


In [81]:
# --- 3. ¿Qué artista tiene más oyentes? ---

query3 = """
SELECT nombre, oyentes 
FROM artistas 
ORDER BY oyentes DESC 
LIMIT 1;
"""
print("\n3. Artista con más oyentes:")
print(pd.read_sql(query3, conexion))


3. Artista con más oyentes:


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\3233166397.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query3, conexion))


     nombre  oyentes
0  Coldplay  8888778


In [82]:
# --- 4. ¿Qué artista ha estado activo durante más años? ---

# Calculamos la diferencia entre el año de su última y primera canción

query4 = """
SELECT a.nombre, (MAX(c.anio) - MIN(c.anio)) AS años_trayectoria
FROM artistas a
JOIN canciones c ON a.id_artista = c.id_artista
GROUP BY a.nombre
ORDER BY años_trayectoria DESC
LIMIT 1;
"""
print("\n4. Artista con la trayectoria más larga:")
print(pd.read_sql(query4, conexion))


4. Artista con la trayectoria más larga:


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\4079707164.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query4, conexion))


     nombre  años_trayectoria
0  Endymion                15


In [83]:
# --- 5. ¿Qué artistas tienen artistas_similares más comunes? ---

# Esta query agrupa a los artistas que comparten exactamente las mismas recomendaciones

query5 = """
SELECT artistas_similares, COUNT(*) AS frecuencia, GROUP_CONCAT(nombre SEPARATOR ' / ') AS artistas_del_grupo
FROM artistas
WHERE artistas_similares <> '[]'
GROUP BY artistas_similares
HAVING frecuencia > 1
ORDER BY frecuencia DESC;
"""
print("\n5. Grupos de artistas con 'similares' comunes:")
print(pd.read_sql(query5, conexion))


5. Grupos de artistas con 'similares' comunes:


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2370412458.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  print(pd.read_sql(query5, conexion))


                                     artistas_similares  frecuencia  \
0     ['WOOZI', 'JxW', 'SEUNGKWAN', 'THE 8', 'DOKYEOM']          16   
1     ['Yodelice', 'Cocoon', 'Cats on Trees', 'Lilly...          16   
2     ['KennyHoopla', 'Algernon Cadwallader', 'Snowi...          16   
3     ['Kukon', 'Paluch', 'Chada', 'Pezet', 'Скрипто...          16   
4     ['LUIS GABRIEL', 'Jador', 'Liviu Guta', 'Bogda...          16   
...                                                 ...         ...   
5524  ['Prem Dhillon', 'Navaan Sandhu', 'Sidhu Moose...           8   
5525  ['Preoccupations', 'Ought', 'Shame', 'Iceage',...           8   
5526  ['Preslava', 'Anelia', 'Desi Slava', 'Toni Sto...           8   
5527  ['j-hope', 'Jin', 'Agust D', 'RM', 'BTS, Halsey']           2   
5528  ['Sabrina Carpenter', 'Olivia Rodrigo', 'Graci...           2   

                                     artistas_del_grupo  
0     Seventeen / SEVENTEEN / Seventeen / SEVENTEEN ...  
1     Aaron / AaRON / AaRON / A

In [84]:
# Canciones 

# --- 1. ¿Cuál es la canción más reciente más popular (2025)? ---

query_reciente = """
SELECT c.nombre AS cancion, a.nombre AS artista, a.num_reproducciones, c.anio
FROM canciones c
JOIN artistas a ON c.id_artista = a.id_artista
WHERE c.anio = 2025
ORDER BY a.num_reproducciones DESC
LIMIT 1
"""

print("Canción más popular de 2025 (Gen Z):")
df_reciente = pd.read_sql(query_reciente, conexion)
display(df_reciente)

Canción más popular de 2025 (Gen Z):


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\232730502.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reciente = pd.read_sql(query_reciente, conexion)


,cancion,artista,num_reproducciones,anio
0,The Fate of Ophelia,Taylor Swift,3497593576,2025


In [85]:
# --- 2. Himnos de cada generación (2010, 2015, 2020, 2025) ---

query_himnos = """
SELECT anio, cancion, artista, num_reproducciones
FROM (
    SELECT c.anio, c.nombre AS cancion, a.nombre AS artista, a.num_reproducciones,
           ROW_NUMBER() OVER(PARTITION BY c.anio ORDER BY a.num_reproducciones DESC) as ranking
    FROM canciones c
    JOIN artistas a ON c.id_artista = a.id_artista
    WHERE c.anio IN (2010, 2015, 2020, 2025)
) as t
WHERE ranking = 1
"""
print("--- Himnos Generacionales ---")
display(pd.read_sql(query_himnos, conexion))

--- Himnos Generacionales ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\3249118634.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query_himnos, conexion))


,anio,cancion,artista,num_reproducciones
0,2010,Mean,Taylor Swift,3497593576
1,2015,Run,BTS,2626063416
2,2020,Life Goes On,BTS,2626063416
3,2025,The Fate of Ophelia,Taylor Swift,3497593576


In [86]:
# --- 3. Clásicos que nunca fallan (aparecen en más de un año destacado) ---

query_clasicos = """
SELECT nombre, COUNT(DISTINCT anio) as total_anios, GROUP_CONCAT(DISTINCT anio) as años_que_aparece
FROM canciones
GROUP BY nombre
HAVING total_anios > 1
ORDER BY total_anios DESC
LIMIT 10
"""
print("\n--- Clásicos Inoxidables ---")
display(pd.read_sql(query_clasicos, conexion))


--- Clásicos Inoxidables ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2593536558.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query_clasicos, conexion))


,nombre,total_anios,años_que_aparece
0,Bleed,4,"2010,2015,2020,2025"
1,CLOSER,4,"2010,2015,2020,2025"
2,Forever,4,"2010,2015,2020,2025"
3,home,4,"2010,2015,2020,2025"
4,honey,4,"2010,2015,2020,2025"
5,LONELY,4,"2010,2015,2020,2025"
6,Magic,4,"2010,2015,2020,2025"
7,Stranger,4,"2010,2015,2020,2025"
8,tell me,4,"2010,2015,2020,2025"
9,Afterglow,3,"2015,2020,2025"


In [87]:
# Géneros y Público

#--- Análisis de Géneros (Peso, Reproducciones y Equilibrio) ---

query_analisis_generos = """
SELECT g.nombre_genero, 
       COUNT(c.id_cancion) AS total_canciones, 
       SUM(a.num_reproducciones) AS total_reproducciones,
       SUM(a.oyentes) AS total_oyentes
FROM generos g
JOIN canciones c ON g.id_genero = c.id_genero
JOIN artistas a ON c.id_artista = a.id_artista
GROUP BY g.nombre_genero
ORDER BY total_reproducciones DESC
"""
print("\n--- Análisis de Géneros (Variedad vs Popularidad) ---")
df_gen_completo = pd.read_sql(query_analisis_generos, conexion)
display(df_gen_completo)


--- Análisis de Géneros (Variedad vs Popularidad) ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\308731184.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_gen_completo = pd.read_sql(query_analisis_generos, conexion)


,nombre_genero,total_canciones,total_reproducciones,total_oyentes
0,pop,23698,1.776592e+12,1.707864e+10
1,rock,23970,1.254526e+12,2.138943e+10
2,indie,23964,1.086400e+12,2.066148e+10
3,punk,23958,5.898061e+11,1.042749e+10


In [88]:
# --- Versatilidad: Artistas populares en géneros distintos ---

query_versatilidad = """
SELECT a.nombre, COUNT(DISTINCT c.id_genero) as total_generos
FROM artistas a
JOIN canciones c ON a.id_artista = c.id_artista
GROUP BY a.nombre
HAVING total_generos > 1
ORDER BY total_generos DESC
LIMIT 10
"""
print("\n--- Artistas más Versátiles ---")
display(pd.read_sql(query_versatilidad, conexion))

# --- Género que domina en cada año clave ---
query_genero_dominante = """
SELECT anio, nombre_genero, total_canciones
FROM (
    SELECT c.anio, g.nombre_genero, COUNT(*) as total_canciones,
           RANK() OVER(PARTITION BY c.anio ORDER BY COUNT(*) DESC) as rnk
    FROM canciones c
    JOIN generos g ON c.id_genero = g.id_genero
    WHERE c.anio IN (2010, 2015, 2020, 2025)
    GROUP BY c.anio, g.nombre_genero
) t
WHERE rnk = 1
"""
print("\n--- Evolución de Gustos Generacionales ---")
display(pd.read_sql(query_genero_dominante, conexion))


--- Artistas más Versátiles ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2838137240.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query_versatilidad, conexion))


,nombre,total_generos
0,Jonna Fraser,4
1,12genresJesus,3
2,24kGoldn,3
3,A Flock Of Seagulls,3
4,All Time Low,3
5,Anne Clark,3
6,Arcade Fire,3
7,Ares,3
8,Bad Boys Blue,3
9,Beach Bunny,3



--- Evolución de Gustos Generacionales ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2838137240.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql(query_genero_dominante, conexion))


,anio,nombre_genero,total_canciones
0,2010,indie,6000
1,2010,rock,6000
2,2015,punk,6000
3,2015,indie,6000
4,2020,punk,6000
5,2020,indie,6000
6,2025,punk,5982
7,2025,rock,5982


In [89]:
# --- Selección de artistas para el evento ---


print("\n--- PROPUESTA DE ORGANIZACIÓN DEL EVENTO ---")

# Abrir: El artista con más oyentes (para que todos se sientan cómodos al llegar)
abrir = pd.read_sql("SELECT nombre, oyentes FROM artistas ORDER BY oyentes DESC LIMIT 1", conexion)

# Mantener: El artista con más variedad/canciones (para que la pista nunca esté vacía)
mantener = pd.read_sql("""
    SELECT a.nombre, COUNT(c.id_cancion) as total 
    FROM artistas a JOIN canciones c ON a.id_artista = c.id_artista 
    GROUP BY a.nombre ORDER BY total DESC LIMIT 1
""", conexion)

# Cerrar: El artista con más reproducciones (el éxito total para terminar por todo lo alto)
cerrar = pd.read_sql("SELECT nombre, num_reproducciones FROM artistas ORDER BY num_reproducciones DESC LIMIT 1", conexion)

print(f"PARA ABRIR: {abrir.iloc[0]['nombre']} (Atrae al máximo público con sus oyentes)")
print(f"PARA MANTENER: {mantener.iloc[0]['nombre']} (Garantiza variedad con su amplio catálogo)")
print(f"PARA CERRAR: {cerrar.iloc[0]['nombre']} (Máxima energía basada en reproducciones totales)")

# FINALMENTE CERRAMOS LA CONEXIÓN
conexion.close()
print("\n✅ Análisis finalizado y conexión cerrada con seguridad.")


--- PROPUESTA DE ORGANIZACIÓN DEL EVENTO ---


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2525374485.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  abrir = pd.read_sql("SELECT nombre, oyentes FROM artistas ORDER BY oyentes DESC LIMIT 1", conexion)
C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2525374485.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mantener = pd.read_sql("""


PARA ABRIR: Coldplay (Atrae al máximo público con sus oyentes)
PARA MANTENER: Tame Impala (Garantiza variedad con su amplio catálogo)
PARA CERRAR: Taylor Swift (Máxima energía basada en reproducciones totales)

✅ Análisis finalizado y conexión cerrada con seguridad.


C:\Users\mmart\AppData\Local\Temp\ipykernel_26696\2525374485.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cerrar = pd.read_sql("SELECT nombre, num_reproducciones FROM artistas ORDER BY num_reproducciones DESC LIMIT 1", conexion)
